# Project Solution - Primary

You have to work on the [Kiva](https://drive.google.com/file/d/1-tJtnIbo1Rt-F1XfoWGVkmBXiI-ciuRx/view) dataset. Some information on the datasets are on the [Kaggle](https://www.kaggle.com/gaborfodor/additional-kiva-snapshot) web page.

## Basic tasks

**1.** Normalize the `loan_lenders` table. In the normalized table, each row must have one `loan_id` and one `lender`.

Firstly, csv file is read and data frame with unnormalized data is created:

In [1]:
import pandas as pd
import numpy as np
import time as tm
FINAL = False # This is final calculation, on complete data
start_time = tm.time()
loans_lenders_un = pd.read_csv('../../Datasets/kiva/loans_lenders.csv')
if not FINAL:
    loans_lenders_un = loans_lenders_un.head(10000) 
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans_lenders_un

--- 3.168536901473999 seconds ---


,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."
...,...,...
9995,45940,"helga4707, james6963, jimjams, andreas2382, si..."
9996,247491,"priyaram, christian9832, john9242, sandra1434,..."
9997,345274,"priyaram, nicola1093, bobby9744, simon7848, di..."
9998,125945,"joseph1859, matt5349, reese3555, stanley3312, ..."


Data frame with structured data will be created with help of intermediate object - list of pairs (loan_id, lender) packed in the dictionary:

In [2]:
import time as tm
start_time = tm.time()
lis = []
for index, row in loans_lenders_un.iterrows(): 
    ls = row['lenders'].split(',')
    for l in ls:
        lis.append({ 'loan_id' : row['loan_id'], 'lender': l.strip() })
loans_lender = pd.DataFrame(lis) 
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans_lender 

--- 2.30903959274292 seconds ---


,loan_id,lender
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
245000,225434,wongacom3393
245001,225434,marleneanddel8151
245002,225434,joanne4956
245003,225434,juddie7070


**2.** For each loan, add a column `duration` corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

Firstly, data frame should be loaded and structure of the data frame `loans` should be determined:

In [3]:
import pandas as pd
import numpy as np
import time as tm
start_time = tm.time()
loans = pd.read_csv('../../Datasets/kiva/loans.csv')
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans.columns

--- 23.96799325942993 seconds ---


Index(['loan_id', 'loan_name', 'original_language', 'description',
       'description_translated', 'funded_amount', 'loan_amount', 'status',
       'activity_name', 'sector_name', 'loan_use', 'country_code',
       'country_name', 'town_name', 'currency_policy',
       'currency_exchange_coverage_rate', 'currency', 'partner_id',
       'posted_time', 'planned_expiration_time', 'disburse_time',
       'raised_time', 'lender_term', 'num_lenders_total',
       'num_journal_entries', 'num_bulk_entries', 'tags', 'borrower_genders',
       'borrower_pictured', 'repayment_interval', 'distribution_model'],
      dtype='object')

After thar, values from the data frame should be displayed:

In [4]:
if not FINAL:
    loans = loans.head(20000)
loans

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,341198,Guillermo Antonio,Spanish,"El Señor Guillermo tiene 42 años, la ubicación...",Guillermo is 42 years old. His business is lo...,375.0,375.0,funded,Shoe Sales,Retail,...,2011-10-13 00:00:41.000 +0000,14.0,10,1,1,NaN,male,true,monthly,field_partner
19996,341771,ERLINDA,English,Erlinda is from the village of Sillawit. She i...,NaN,175.0,175.0,funded,General Store,Retail,...,2011-10-14 06:02:23.000 +0000,8.0,3,2,1,NaN,female,true,irregular,field_partner
19997,344237,Ma Guadalupe,Spanish,La señora Ma Guadalupe es madre de 2 hijos de ...,\r\nMaría Guadalupe is the mother of two child...,625.0,625.0,funded,Property,Housing,...,2011-10-27 19:23:44.000 +0000,14.0,23,1,1,NaN,female,true,monthly,field_partner
19998,344735,Blanca Lidia,Spanish,"Blanca, vive con su esposo, tiene tres hijos d...",Blanca lives with her husband and has three ch...,350.0,350.0,funded,Food Production/Sales,Food,...,2011-10-12 02:10:27.000 +0000,8.0,10,4,2,NaN,female,true,monthly,field_partner


We are now mainly interested in columns 'planned_expiration_time', 'disburse_time':

In [5]:
loans_sel = loans[ ['loan_id', 'loan_name','planned_expiration_time', 'disburse_time'] ]
loans_sel

,loan_id,loan_name,planned_expiration_time,disburse_time
0,657307,Aivy,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000
1,657259,Idalia Marizza,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000
2,658010,Aasia,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000
3,659347,Gulmira,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000
4,656933,Ricky\t,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000
...,...,...,...,...
19995,341198,Guillermo Antonio,NaN,2011-09-13 07:00:00.000 +0000
19996,341771,ERLINDA,NaN,2011-09-16 07:00:00.000 +0000
19997,344237,Ma Guadalupe,NaN,2011-10-07 07:00:00.000 +0000
19998,344735,Blanca Lidia,NaN,2011-09-16 07:00:00.000 +0000


In [6]:
from datetime import datetime as dt
import time as tm
start_time = tm.time()
loans['duration'] = None
for index, row in loans.iterrows(): 
    s2 = row['planned_expiration_time']
    s1 = row['disburse_time']
    try:
        d2 = dt.strptime(s2, "%Y-%m-%d %H:%M:%S.%f %z")
    except ValueError:
        continue
    except TypeError:
        continue
    try:
        d1 = dt.strptime(s1, "%Y-%m-%d %H:%M:%S.%f %z")
    except ValueError:
        continue
    except TypeError:
        continue
    loans.loc[index,'duration'] = (d2 - d1).days 
loans_sel = loans[ ['loan_id', 'loan_name','planned_expiration_time', 'disburse_time', 'duration'] ]
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans_sel

--- 12.225908041000366 seconds ---


,loan_id,loan_name,planned_expiration_time,disburse_time,duration
0,657307,Aivy,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,53
1,657259,Idalia Marizza,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,96
2,658010,Aasia,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000,37
3,659347,Gulmira,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000,34
4,656933,Ricky\t,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000,57
...,...,...,...,...,...
19995,341198,Guillermo Antonio,NaN,2011-09-13 07:00:00.000 +0000,None
19996,341771,ERLINDA,NaN,2011-09-16 07:00:00.000 +0000,None
19997,344237,Ma Guadalupe,NaN,2011-10-07 07:00:00.000 +0000,None
19998,344735,Blanca Lidia,NaN,2011-09-16 07:00:00.000 +0000,None


**3.** Find the lenders that have funded at least twice.

- Those are lenders that are duplicated in structured loan_leneders dataframe.


In [7]:
import time as tm
start_time = tm.time()
loans_lender_2_more = loans_lender[loans_lender.duplicated(['lender'])]
lenders_funded_2_more = list(set(loans_lender_2_more['lender']))
print("--- %s seconds ---" % (tm.time()-start_time)) 
lenders_funded_2_more

--- 0.07701730728149414 seconds ---


['lamberta9654',
 'howard7173',
 'kai4998',
 'mark9264',
 'petrea7375',
 'ruediger9216',
 'martin8301',
 'tammy4444',
 'michael3732',
 'iridiumtechnology',
 'christopher6828',
 'veronika4220',
 'kathryn7731',
 'landon6474',
 'nomi7715',
 'rectorfriends',
 'dilshad2613',
 'fredrik1826',
 'john1703',
 'stu9472',
 'john2040',
 'richard3791',
 'christopher7489',
 'elvira7075',
 'bernard6333',
 'rotarycluboftampawes2192',
 'dovile9205',
 'chris6568',
 'alireza5588',
 'andreas5608',
 'steve831',
 'claire4629',
 'roger6386',
 'joe6905',
 'justin3174',
 'richard1638',
 'patrick2731',
 'maryellen4166',
 'ruperte',
 'vidar8372',
 'kevin67319182',
 'deydos',
 'denise3007',
 'johaneje',
 'pat5627',
 'reisha8618',
 'kurt12764',
 'jon4126',
 'linda3649',
 'ryan6553',
 'ragnhild9457',
 'matthewloewen',
 'thomas3190',
 'kristina7552',
 'michael6689',
 'gretchen3915',
 'robert3882',
 'thomas45738761',
 'kris49493180',
 'daniel8981',
 'cathy9608',
 'justin2643',
 'hans7665',
 'judy6694',
 'mau',
 'marge

**4.** For each country, compute how many loans have involved that country as borrowers.

- This solution is based on panda grouping:

In [8]:
import time as tm
start_time = tm.time()
loans_count = loans.groupby(['country_code','country_name'], as_index=False)[['loan_id']].count()
loans_count.columns = ['country_code','country_name','loan_id_count']
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans_count

--- 0.006965160369873047 seconds ---


,country_code,country_name,loan_id_count
0,AF,Afghanistan,12
1,AL,Albania,50
2,AM,Armenia,212
3,AZ,Azerbaijan,109
4,BA,Bosnia and Herzegovina,1
...,...,...,...
73,XK,Kosovo,51
74,YE,Yemen,88
75,ZA,South Africa,16
76,ZM,Zambia,18


**5.** For each country, compute the overall amount of money borrowed.


- This solution is based on panda grouping:

In [9]:
import time as tm
start_time = tm.time()
loans_sum = loans.groupby(['country_code','country_name'], as_index = False)[['loan_amount']].sum()
loans_sum.columns = ['country_code','country_name','loan_amount_sum']
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans_sum

--- 0.006963968276977539 seconds ---


,country_code,country_name,loan_amount_sum
0,AF,Afghanistan,11625.0
1,AL,Albania,69800.0
2,AM,Armenia,371075.0
3,AZ,Azerbaijan,205675.0
4,BA,Bosnia and Herzegovina,575.0
...,...,...,...
73,XK,Kosovo,74950.0
74,YE,Yemen,92100.0
75,ZA,South Africa,29000.0
76,ZM,Zambia,32650.0


**6.** Like the previous point, but expressed as a percentage of the overall amount lent.


- This solution is based on panda functions:

In [10]:
import time as tm
start_time = tm.time()
loans_total = loans['loan_amount'].sum()
loans_percent = loans.groupby(['country_code','country_name'], as_index = False)[['loan_amount']].sum()
loans_percent.columns = ['country_code','country_name','loan_amount_sum']
loans_percent['percent'] = loans_percent['loan_amount_sum']/loans_total*100
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans_percent

--- 0.01796555519104004 seconds ---


,country_code,country_name,loan_amount_sum,percent
0,AF,Afghanistan,11625.0,0.069063
1,AL,Albania,69800.0,0.414677
2,AM,Armenia,371075.0,2.204531
3,AZ,Azerbaijan,205675.0,1.221901
4,BA,Bosnia and Herzegovina,575.0,0.003416
...,...,...,...,...
73,XK,Kosovo,74950.0,0.445273
74,YE,Yemen,92100.0,0.547160
75,ZA,South Africa,29000.0,0.172287
76,ZM,Zambia,32650.0,0.193971


**7.** Like the three previous points, but split for each year (with respect to `disburse time`).


- This solution is based on panda functions:

In [11]:
import time as tm
start_time = tm.time()
from datetime import datetime as dt
loans_total = loans['loan_amount'].sum()
loans['disburse_time_year'] = None
for index, row in loans.iterrows():
    try:
        y = dt.strptime(row['disburse_time'], "%Y-%m-%d %H:%M:%S.%f %z").year
    except TypeError:
        y = None
    except ValueError:
        y = None
    loans.loc[index,'disburse_time_year'] = y 
loans_agg = loans.groupby(['country_code','country_name','disburse_time_year'\
                          ], as_index = False).agg({'loan_id':['count'],\
                                                    'loan_amount':['sum', lambda x: x.sum()/loans_total*100]})
loans_agg.columns = ['country_code','country_name','disburse_time_year','loan_id_count', \
                     'loan_amount_sum','loan_amount_percent']
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans_agg

--- 17.738855600357056 seconds ---


,country_code,country_name,disburse_time_year,loan_id_count,loan_amount_sum,loan_amount_percent
0,AF,Afghanistan,2009,11,10450.0,0.062083
1,AF,Afghanistan,2010,1,1175.0,0.006981
2,AL,Albania,2012,5,4250.0,0.025249
3,AL,Albania,2013,15,19150.0,0.113769
4,AL,Albania,2014,16,27225.0,0.161742
...,...,...,...,...,...,...
514,ZW,Zimbabwe,2013,9,16450.0,0.097728
515,ZW,Zimbabwe,2014,47,31200.0,0.185357
516,ZW,Zimbabwe,2015,9,9150.0,0.054360
517,ZW,Zimbabwe,2016,7,8950.0,0.053171


**8.** For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.


- This solution is based on panda functions:

In [12]:
import time as tm
start_time = tm.time()
merged = pd.merge(left=loans_lender,right=loans, left_on='loan_id', right_on='loan_id')
merged = merged[['loan_id','lender', 'loan_amount']]
number_of_lenders_by_loan_id = merged.groupby(['loan_id'], as_index = False)[['lender']].count()
number_of_lenders_by_loan_id
merged2 = pd.merge(left=merged,right=number_of_lenders_by_loan_id, left_on='loan_id', right_on='loan_id')
merged2['precise_amount'] = merged2.loan_amount / merged2.lender_y
merged2
lent_by_lender = merged2.groupby(['lender_x'], as_index = False)[['precise_amount']].sum()
print("--- %s seconds ---" % (tm.time()-start_time)) 
lent_by_lender
#result.loc[result['lender_x'] == 'muc888'] # result for one lender

--- 0.11765146255493164 seconds ---


,lender_x,precise_amount
0,100ofhumanity1199,64.583333
1,10yearitch,33.876812
2,123321,153.990193
3,1ghostorchid,31.097561
4,1nottus1,43.750000
...,...,...
19663,zubair1627,41.233766
19664,zubin,104.088235
19665,zuzu,25.714286
19666,zxdr,32.500000


**9.** For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.


In [13]:
import pandas as pd
import numpy as np
import time as tm
start_time = tm.time()
lenders = pd.read_csv('../../Datasets/kiva/lenders.csv')
if not FINAL:
    lenders = lenders.head(20000) 
lender_states = lenders[['permanent_name','country_code']]
lender_states

,permanent_name,country_code
0,qian3013,NaN
1,reena6733,NaN
2,mai5982,NaN
3,andrew86079135,NaN
4,nguyen6962,NaN
...,...,...
19995,mohamed4338,NaN
19996,vicrennaisance8567,NaN
19997,mariz6787,NaN
19998,julian2868,NaN


In [14]:
lenders_in_states = lender_states.loc[lender_states['country_code'].notnull()]
lenders_in_states

,permanent_name,country_code
16,naresh2074,US
31,christina27976796,US
37,vikas1098,IN
39,qian1385,US
42,xigg8769,US
...,...,...
19979,manrike5051,LA
19981,janell3482,US
19984,sheryl2462,US
19989,amy5291,US


In [15]:
lenders_without_states = lender_states.loc[lender_states['country_code'].isnull()]
lenders_without_states

,permanent_name,country_code
0,qian3013,NaN
1,reena6733,NaN
2,mai5982,NaN
3,andrew86079135,NaN
4,nguyen6962,NaN
...,...,...
19995,mohamed4338,NaN
19996,vicrennaisance8567,NaN
19997,mariz6787,NaN
19998,julian2868,NaN


In [16]:
lent_and_state = pd.merge(left=lent_by_lender, right=lenders_in_states, left_on='lender_x', right_on='permanent_name')
lent_total = lent_and_state['precise_amount'].sum()
lent_by_state = lent_and_state.groupby(['country_code'], as_index = False\
                                      ).agg({'precise_amount':['count', 'sum', \
                                                               lambda x: x.sum()/lent_total]})
lent_by_state.columns = ['country_code', 'lent_count', 'lent_sum', 'lent_factor']
lent_by_state

,country_code,lent_count,lent_sum,lent_factor
0,AU,1,33.333333,0.431034
1,US,1,44.000000,0.568966


In [17]:
lent_no_state = pd.merge(left=lent_by_lender, right=lenders_without_states, \
                         left_on='lender_x', right_on='permanent_name')
lent_no_state_total = lent_no_state['precise_amount'].sum()
lent_no_state_total

305.0

In [18]:
lent_by_state['additional_lent'] = lent_by_state.lent_factor * lent_no_state_total
lent_by_state

,country_code,lent_count,lent_sum,lent_factor,additional_lent
0,AU,1,33.333333,0.431034,131.465517
1,US,1,44.000000,0.568966,173.534483


In [19]:
loan_and_state = loans[['loan_id','country_code','loan_amount']]
loan_and_state

,loan_id,country_code,loan_amount
0,657307,PH,125.0
1,657259,HN,400.0
2,658010,PK,400.0
3,659347,KG,625.0
4,656933,PH,425.0
...,...,...,...
19995,341198,NI,375.0
19996,341771,PH,175.0
19997,344237,MX,625.0
19998,344735,SV,350.0


In [20]:
loan_by_state = loan_and_state.groupby(['country_code'], as_index = False).agg({'loan_amount':['sum']})
loan_by_state.columns = ['country_code','loan_amount_sum']
loan_by_state

,country_code,loan_amount_sum
0,AF,11625.0
1,AL,69800.0
2,AM,371075.0
3,AZ,205675.0
4,BA,575.0
...,...,...
73,XK,74950.0
74,YE,92100.0
75,ZA,29000.0
76,ZM,32650.0


In [21]:
loan_and_lent = pd.merge(loan_by_state, lent_by_state, how='left', on=['country_code'])
loan_and_lent['difference'] = loan_and_lent.loan_amount_sum - loan_and_lent.lent_sum - loan_and_lent.additional_lent 
print("--- %s seconds ---" % (tm.time()-start_time)) 
loan_and_lent[loan_and_lent.difference == loan_and_lent.difference.max()]

--- 2.8481290340423584 seconds ---


,country_code,loan_amount_sum,lent_count,lent_sum,lent_factor,additional_lent,difference
70,US,294625.0,1.0,44.0,0.568966,173.534483,294407.465517


**10.** Which country has the highest ratio between the difference computed at the previous point and the population?


In [22]:
import pandas as pd
import numpy as np
import time as tm
start_time = tm.time()
country_stat = pd.read_csv('../../Datasets/kiva/country_stats.csv')
country_stat = country_stat[['country_code', 'country_name', 'population', 'population_below_poverty_line']]
country_stat

,country_code,country_name,population,population_below_poverty_line
0,IN,India,1339180127,21.9
1,NG,Nigeria,190886311,70.0
2,MX,Mexico,129163276,46.2
3,PK,Pakistan,197015955,29.5
4,BD,Bangladesh,164669751,31.5
...,...,...,...,...
169,SO,Somalia,14742523,NaN
170,CF,Central African Republic,4659080,NaN
171,WS,Samoa,196440,NaN
172,PS,Palestine,4920724,NaN


In [23]:
country_stat_loan_and_lent = pd.merge(left=country_stat, right=loan_and_lent, \
                                      left_on='country_code', right_on='country_code')
country_stat_loan_and_lent = country_stat_loan_and_lent[['country_code', 'country_name', 'difference', \
                                                         'population', 'population_below_poverty_line']]
country_stat_loan_and_lent['ratio_1'] = country_stat_loan_and_lent.difference/country_stat_loan_and_lent.population
print("--- %s seconds ---" % (tm.time()-start_time)) 
country_stat_loan_and_lent[country_stat_loan_and_lent.ratio_1 == country_stat_loan_and_lent.ratio_1.max()]

--- 0.020999670028686523 seconds ---


,country_code,country_name,difference,population,population_below_poverty_line,ratio_1
5,US,United States,294407.465517,324459463,15.1,0.000907


**11.** Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?


In [24]:
import time as tm
start_time = tm.time()
country_stat_loan_and_lent = pd.merge(left=country_stat, right=loan_and_lent, \
                                      left_on='country_code', right_on='country_code')
country_stat_loan_and_lent = country_stat_loan_and_lent[['country_code', 'country_name', 'difference', \
                                                         'population', 'population_below_poverty_line']]
country_stat_loan_and_lent['ratio_2'] = country_stat_loan_and_lent.difference\
/(country_stat_loan_and_lent.population * ((100-country_stat_loan_and_lent.population_below_poverty_line)/100) )
print("--- %s seconds ---" % (tm.time()-start_time)) 
country_stat_loan_and_lent[country_stat_loan_and_lent.ratio_2 == country_stat_loan_and_lent.ratio_2.max()]

--- 0.006001949310302734 seconds ---


,country_code,country_name,difference,population,population_below_poverty_line,ratio_2
5,US,United States,294407.465517,324459463,15.1,0.001069


**12.** For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [25]:
import time as tm
start_time = tm.time()

from datetime import datetime as dt, timezone as tz

now = dt.now(tz.utc)

def days_in_year(year):
    d1 = dt.strptime( str(year)+'-01-01', "%Y-%m-%d") 
    d1 = d1.replace(tzinfo=None)
    d2 = dt.strptime( str(year)+'-12-31', "%Y-%m-%d")
    d2 = d2.replace(tzinfo=None)
    return (d2-d1).days+1

def total_amount_of_loans(disb, plan_exp, amount):
    if( disb.year == plan_exp.year ):
        return [(disb.year,amount)]
    disb = disb.replace(tzinfo=None)
    plan_exp = plan_exp.replace(tzinfo=None) 
    total_days = (plan_exp-disb).days 
    total_days += 1
    total_amount = []
    curr = disb
    curr = curr.replace(tzinfo=None)
    for y in range(disb.year, plan_exp.year):
        dy = (dt.strptime(str(y)+'-12-31',"%Y-%m-%d").replace(tzinfo=None)-curr).days + 1
        total_amount.append((y, dy * amount / total_days))
        curr = dt.strptime(str(y+1)+'-01-01',"%Y-%m-%d")
        curr = curr.replace(tzinfo=None)
    dy = (plan_exp-curr).days + 1
    total_amount.append( (plan_exp.year, dy * amount / total_days) )
    return total_amount

d_t = dt.strptime( '2016-12-01', "%Y-%m-%d")
p_e_t = dt.strptime( '2018-01-30', "%Y-%m-%d")
total_amount_of_loans(d_t, p_e_t, 5000)

[(2016, 363.84976525821594),
 (2017, 4284.037558685446),
 (2018, 352.11267605633805)]

In [26]:
import pandas as pd
import numpy as np
loans
loans_base = loans[['loan_id','loan_name','loan_amount','planned_expiration_time','disburse_time']]
loans_base

,loan_id,loan_name,loan_amount,planned_expiration_time,disburse_time
0,657307,Aivy,125.0,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000
1,657259,Idalia Marizza,400.0,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000
2,658010,Aasia,400.0,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000
3,659347,Gulmira,625.0,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000
4,656933,Ricky\t,425.0,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000
...,...,...,...,...,...
19995,341198,Guillermo Antonio,375.0,NaN,2011-09-13 07:00:00.000 +0000
19996,341771,ERLINDA,175.0,NaN,2011-09-16 07:00:00.000 +0000
19997,344237,Ma Guadalupe,625.0,NaN,2011-10-07 07:00:00.000 +0000
19998,344735,Blanca Lidia,350.0,NaN,2011-09-16 07:00:00.000 +0000


In [28]:
lis = []
for index, row in loans_base.iterrows():
    s2 = row['planned_expiration_time']
    try:
        d2 = dt.strptime(s2, "%Y-%m-%d %H:%M:%S.%f %z")
    except ValueError:
        continue
    except TypeError:
        continue
    s1 = row['disburse_time']
    try:
        d1 = dt.strptime(s1, "%Y-%m-%d %H:%M:%S.%f %z")
    except ValueError:
        continue
    except TypeError:
        continue
    try:
        amt = float(row['loan_amount'])
    except ValueError:
        continue
    except TypeError:
        continue
    ls = total_amount_of_loans(d1, d2, amt)
    for l in ls:
        lis.append({ 'loan_id' : row['loan_id'], 'loan_name': row['loan_name'], 'loan_amount': row['loan_amount'], \
                    'planned_expiration_time': row['planned_expiration_time'], \
                    'disburse_time': row['disburse_time'],'year': l[0], 'ammount':l[1] })
loans_final = pd.DataFrame(lis) 
print("--- %s seconds ---" % (tm.time()-start_time)) 
loans_final

--- 203.29502081871033 seconds ---


,loan_id,loan_name,loan_amount,planned_expiration_time,disburse_time,year,ammount
0,657307,Aivy,125.0,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,2013,20.833333
1,657307,Aivy,125.0,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,2014,104.166667
2,657259,Idalia Marizza,400.0,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,2013,45.360825
3,657259,Idalia Marizza,400.0,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,2014,350.515464
4,658010,Aasia,400.0,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000,2014,400.000000
...,...,...,...,...,...,...,...
19962,971438,CHHON,1300.0,2015-12-01 02:00:03.000 +0000,2015-10-02 07:00:00.000 +0000,2015,1300.000000
19963,1267448,Khen's Group,100.0,2017-05-01 18:50:04.000 +0000,2017-03-21 07:00:00.000 +0000,2017,100.000000
19964,1268098,Hieu,900.0,2017-05-17 19:50:05.000 +0000,2017-03-22 07:00:00.000 +0000,2017,900.000000
19965,1268162,Hussain,2000.0,2017-05-02 22:20:02.000 +0000,2017-03-21 07:00:00.000 +0000,2017,2000.000000
